In [ ]:
#Install tqdm to have progress bar
!pip install tqdm

#install necessary pkg to make connection with elasticsearch domain
!pip install requests
!pip install requests-aws4auth
!pip install "sagemaker>=2.0.0<3.0.0"

In [ ]:
# %%capture
# import IPython
# import sys

# !{sys.executable} -m pip install ipywidgets
# IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

In [ ]:
import boto3
import re
import time
import os
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

In [ ]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "sbert"

outputs = get_cfn_outputs(cloudformation_stack_name)

backend_bucket = outputs['s3BucketBackend']

outputs

## Deploying a full-stack NLU search application

In [ ]:
s3_resource.Object(backend_bucket, 'backend/template.yaml').upload_file('./backend/template.yaml', ExtraArgs={'ACL':'public-read'})

sam_template_url = f'https://{backend_bucket}.s3.amazonaws.com/backend/template.yaml'

# Generate the CloudFormation Quick Create Link

print("Click the URL below to create the backend API for sbert:\n")
print((
    'https://console.aws.amazon.com/cloudformation/home?region=eu-central-1#/stacks/create/review'
    f'?templateURL={sam_template_url}'
    '&stackName=sbert-api'
    f'&param_BucketBackend={outputs["s3BucketBackend"]}'
))

In [ ]:
import json
api_endpoint = get_cfn_outputs('sbert-api')['TextSimilarityApi']

api_endpoint

## Cleanup

Make sure that you stop the notebook instance and delete the Amazon SageMaker endpoint.

In [ ]:
# Empty S3 Contents
bucket_resource = s3_resource.Bucket(backend_bucket)
bucket_resource.objects.all().delete()